In [1]:
import numpy as np
import pandas as pd
import pytest
from sqlalchemy import create_engine

In [5]:
df = pd.read_csv('../../data/raw/national-park-trails.csv')
df.head(3)

,trail_id,name,area_name,city_name,state_name,country_name,_geoloc,popularity,length,elevation_gain,difficulty_rating,route_type,visitor_usage,avg_rating,num_reviews,features,activities,units
0,10020048,Harding Ice Field Trail,Kenai Fjords National Park,Seward,Alaska,United States,"{'lat': 60.18852, 'lng': -149.63156}",24.8931,15610.598,1161.8976,5,out and back,3.0,5.0,423,"['dogs-no', 'forest', 'river', 'views', 'water...","['birding', 'camping', 'hiking', 'nature-trips...",i
1,10236086,Mount Healy Overlook Trail,Denali National Park,Denali National Park,Alaska,United States,"{'lat': 63.73049, 'lng': -148.91968}",18.0311,6920.162,507.7968,3,out and back,1.0,4.5,260,"['dogs-no', 'forest', 'views', 'wild-flowers',...","['birding', 'camping', 'hiking', 'nature-trips...",i
2,10267857,Exit Glacier Trail,Kenai Fjords National Park,Seward,Alaska,United States,"{'lat': 60.18879, 'lng': -149.631}",17.7821,2896.812,81.9912,1,out and back,3.0,4.5,224,"['dogs-no', 'partially-paved', 'views', 'wildl...","['hiking', 'walking']",i


In [6]:
np.shape(df)

(3313, 18)

# Read from SQL

In [7]:
# # Connect to database, replace with RDS string in script
# cn = create_engine('sqlite:///../../data/trails.db').connect()
  
# # Read as dataframe
# df = pd.read_sql_table('trails', cn)

# Convert `length` units

In [8]:
def yards_to_miles(df, column_name, digits):
    df[column_name] = round(df[column_name] / 1760, digits)
    return df

# Remove closed trails

In [9]:
def df_drop_str(df, column_name, drop_str):
    df_drop = df[~df[column_name].str.lower().str.contains(drop_str)]
    return df_drop

# Parse location
Parse latitude and longitude from `_geoloc` column.

In [8]:
# def parse_one_geoloc(geoloc):
    
#     # Remove parenthesis and only grab latitude and longitude
#     split_ls = geoloc.strip('{}').split(',')
#     geoloc_ls = [float(i.split(':')[1].strip()) for i in split_ls]
    
#     return geoloc_ls

# def parse_geoloc(df, column_name):
    
#     # Create new columns for latitude and longitude
#     geoloc_ls = [parse_one_geoloc(i) for i in df[column_name]]
#     df['lat'] = [i[0] for i in geoloc_ls]
#     df['lng'] = [i[1] for i in geoloc_ls]
    
#     # Drop the original variable
#     df = df.drop(columns=column_name)
    
#     return df

# Create cleaned dataset

In [8]:
# yards to miles
column_name1 = 'length'
digits = 2

# drop closed trails
column_name2 = 'name'
drop_str = 'closed'

In [9]:
# Convert yards to miles
df_convert = yards_to_miles(df, column_name1, digits)

# Drop trails
df_clean = df_drop_str(df_convert, column_name2, drop_str)

# Parse geoloc
# df_geoloc = parse_geoloc(df, '_geoloc')
# df_geoloc.to_csv('../../data/models/df_cleaned.csv', index=False)

In [12]:
df_clean.head()
#df_clean[['trail_id','name','area_name','city_name','length','elevation_gain','route_type','features','activities']]

,trail_id,name,area_name,city_name,length,elevation_gain,route_type,features,activities
0,10020048,Harding Ice Field Trail,Kenai Fjords National Park,Seward,8.87,1161.8976,out and back,"['dogs-no', 'forest', 'river', 'views', 'water...","['birding', 'camping', 'hiking', 'nature-trips..."
1,10236086,Mount Healy Overlook Trail,Denali National Park,Denali National Park,3.93,507.7968,out and back,"['dogs-no', 'forest', 'views', 'wild-flowers',...","['birding', 'camping', 'hiking', 'nature-trips..."
2,10267857,Exit Glacier Trail,Kenai Fjords National Park,Seward,1.65,81.9912,out and back,"['dogs-no', 'partially-paved', 'views', 'wildl...","['hiking', 'walking']"
3,10236076,Horseshoe Lake Trail,Denali National Park,Denali National Park,1.92,119.7864,loop,"['dogs-no', 'forest', 'lake', 'kids', 'views',...","['birding', 'hiking', 'nature-trips', 'trail-r..."
4,10236082,Triple Lakes Trail,Denali National Park,Denali National Park,16.92,1124.7120,out and back,"['dogs-no', 'lake', 'views', 'wild-flowers', '...","['birding', 'fishing', 'hiking', 'nature-trips..."
...,...,...,...,...,...,...,...,...,...
3307,10236006,Pa ka'oao Trail (White Hill Trail),Haleakala National Park,Kula,0.46,32.9184,out and back,"['dogs-no', 'kids', 'views']","['hiking', 'walking']"
3308,10008302,Silversword Loop Via Halemau'u Trail,Haleakala National Park,Kula,11.43,1105.8144,loop,"['dogs-no', 'views', 'wild-flowers']","['birding', 'hiking', 'nature-trips']"
3309,10236001,Keonehe'ehe'e Trail,Haleakala National Park,Kula,16.09,1171.9560,out and back,"['dogs-no', 'views', 'wildlife']","['backpacking', 'camping', 'hiking']"
3310,10258707,Red Hill Overlook Summit Trail,Haleakala National Park,Kula,0.18,3.9624,out and back,"['dogs-no', 'kids', 'views']","['hiking', 'walking']"


In [11]:
np.shape(df_clean)

(3294, 18)

# Create test input

In [16]:
data_subset = df.sample(3, random_state=0)
data_subset

,trail_id,name,area_name,city_name,state_name,country_name,_geoloc,popularity,length,elevation_gain,difficulty_rating,route_type,visitor_usage,avg_rating,num_reviews,features,activities,units
1698,10008848,Baxter Creek Trail,Great Smoky Mountains National Park,Waynesville,North Carolina,United States,"{'lat': 35.75111, 'lng': -83.10966}",4.9204,25266.638,1522.7808,5,point to point,2.0,4.5,7,"['dogs-no', 'forest', 'views', 'wild-flowers']","['birding', 'hiking', 'nature-trips']",i
2919,10032315,Sunnybrook Meadows Trail,Olympic National Park,Brinnon,Washington,United States,"{'lat': 47.72974, 'lng': -123.14179}",3.0804,26393.176,2102.8152,7,out and back,2.0,5.0,1,"['dogs-no', 'views', 'wild-flowers', 'wildlife']","['backpacking', 'birding', 'camping', 'hiking'...",i
224,10027503,Clouds Rest Trail via Tenaya Lake,Yosemite National Park,Yosemite Valley,California,United States,"{'lat': 37.82585, 'lng': -119.47046}",38.5795,19794.882,948.8424,5,out and back,3.0,5.0,685,"['dogs-no', 'forest', 'lake', 'views', 'wild-f...","['backpacking', 'birding', 'camping', 'hiking'...",i


In [17]:
data_subset.values

array([[10008848, 'Baxter Creek Trail',
        'Great Smoky Mountains National Park', 'Waynesville',
        'North Carolina', 'United States',
        "{'lat': 35.75111, 'lng': -83.10966}", 4.9204, 25266.638,
        1522.7808, 5, 'point to point', 2.0, 4.5, 7,
        "['dogs-no', 'forest', 'views', 'wild-flowers']",
        "['birding', 'hiking', 'nature-trips']", 'i'],
       [10032315, 'Sunnybrook Meadows Trail', 'Olympic National Park',
        'Brinnon', 'Washington', 'United States',
        "{'lat': 47.72974, 'lng': -123.14179}", 3.0804, 26393.176,
        2102.8152, 7, 'out and back', 2.0, 5.0, 1,
        "['dogs-no', 'views', 'wild-flowers', 'wildlife']",
        "['backpacking', 'birding', 'camping', 'hiking', 'nature-trips']",
        'i'],
       [10027503, 'Clouds Rest Trail via Tenaya Lake',
        'Yosemite National Park', 'Yosemite Valley', 'California',
        'United States', "{'lat': 37.82585, 'lng': -119.47046}", 38.5795,
        19794.882, 948.8424, 5, 'out an

In [18]:
data_subset.index

Int64Index([1698, 2919, 224], dtype='int64')

In [19]:
data_subset.columns

Index(['trail_id', 'name', 'area_name', 'city_name', 'state_name',
       'country_name', '_geoloc', 'popularity', 'length', 'elevation_gain',
       'difficulty_rating', 'route_type', 'visitor_usage', 'avg_rating',
       'num_reviews', 'features', 'activities', 'units'],
      dtype='object')

In [20]:
data_subset.index.name

In [21]:
data_subset.columns.name

In [22]:
df_in_values = [[10008848, 'Baxter Creek Trail',
        'Great Smoky Mountains National Park', 'Waynesville',
        'North Carolina', 'United States',
        "{'lat': 35.75111, 'lng': -83.10966}", 4.9204, 25266.638,
        1522.7808, 5, 'point to point', 2.0, 4.5, 7,
        "['dogs-no', 'forest', 'views', 'wild-flowers']",
        "['birding', 'hiking', 'nature-trips']", 'i'],
       [10032315, 'Sunnybrook Meadows Trail', 'Olympic National Park',
        'Brinnon', 'Washington', 'United States',
        "{'lat': 47.72974, 'lng': -123.14179}", 3.0804, 26393.176,
        2102.8152, 7, 'out and back', 2.0, 5.0, 1,
        "['dogs-no', 'views', 'wild-flowers', 'wildlife']",
        "['backpacking', 'birding', 'camping', 'hiking', 'nature-trips']",
        'i'],
       [10027503, 'Clouds Rest Trail via Tenaya Lake',
        'Yosemite National Park', 'Yosemite Valley', 'California',
        'United States', "{'lat': 37.82585, 'lng': -119.47046}", 38.5795,
        19794.882, 948.8424, 5, 'out and back', 3.0, 5.0, 685,
        "['dogs-no', 'forest', 'lake', 'views', 'wild-flowers', 'wildlife']",
        "['backpacking', 'birding', 'camping', 'hiking', 'nature-trips', 'trail-running', 'horseback-riding']",
        'i']]
df_in_index = [1698, 2919, 224]
df_in_columns = ['trail_id', 'name', 'area_name', 'city_name', 'state_name',
       'country_name', '_geoloc', 'popularity', 'length', 'elevation_gain',
       'difficulty_rating', 'route_type', 'visitor_usage', 'avg_rating',
       'num_reviews', 'features', 'activities', 'units']
df_in = pd.DataFrame(df_in_values, index=df_in_index, columns=df_in_columns)

In [23]:
df_in.equals(data_subset)

True

## Create tests for `yards_to_miles()`

In [24]:
df_out = yards_to_miles(df_in, 'length', 2)
df_out

,trail_id,name,area_name,city_name,state_name,country_name,_geoloc,popularity,length,elevation_gain,difficulty_rating,route_type,visitor_usage,avg_rating,num_reviews,features,activities,units
1698,10008848,Baxter Creek Trail,Great Smoky Mountains National Park,Waynesville,North Carolina,United States,"{'lat': 35.75111, 'lng': -83.10966}",4.9204,14.36,1522.7808,5,point to point,2.0,4.5,7,"['dogs-no', 'forest', 'views', 'wild-flowers']","['birding', 'hiking', 'nature-trips']",i
2919,10032315,Sunnybrook Meadows Trail,Olympic National Park,Brinnon,Washington,United States,"{'lat': 47.72974, 'lng': -123.14179}",3.0804,15.00,2102.8152,7,out and back,2.0,5.0,1,"['dogs-no', 'views', 'wild-flowers', 'wildlife']","['backpacking', 'birding', 'camping', 'hiking'...",i
224,10027503,Clouds Rest Trail via Tenaya Lake,Yosemite National Park,Yosemite Valley,California,United States,"{'lat': 37.82585, 'lng': -119.47046}",38.5795,11.25,948.8424,5,out and back,3.0,5.0,685,"['dogs-no', 'forest', 'lake', 'views', 'wild-f...","['backpacking', 'birding', 'camping', 'hiking'...",i


In [26]:
df_out.values

array([[10008848, 'Baxter Creek Trail',
        'Great Smoky Mountains National Park', 'Waynesville',
        'North Carolina', 'United States',
        "{'lat': 35.75111, 'lng': -83.10966}", 4.9204, 14.36, 1522.7808,
        5, 'point to point', 2.0, 4.5, 7,
        "['dogs-no', 'forest', 'views', 'wild-flowers']",
        "['birding', 'hiking', 'nature-trips']", 'i'],
       [10032315, 'Sunnybrook Meadows Trail', 'Olympic National Park',
        'Brinnon', 'Washington', 'United States',
        "{'lat': 47.72974, 'lng': -123.14179}", 3.0804, 15.0, 2102.8152,
        7, 'out and back', 2.0, 5.0, 1,
        "['dogs-no', 'views', 'wild-flowers', 'wildlife']",
        "['backpacking', 'birding', 'camping', 'hiking', 'nature-trips']",
        'i'],
       [10027503, 'Clouds Rest Trail via Tenaya Lake',
        'Yosemite National Park', 'Yosemite Valley', 'California',
        'United States', "{'lat': 37.82585, 'lng': -119.47046}", 38.5795,
        11.25, 948.8424, 5, 'out and back', 3.0,

In [27]:
df_out.index

Int64Index([1698, 2919, 224], dtype='int64')

In [28]:
df_out.columns

Index(['trail_id', 'name', 'area_name', 'city_name', 'state_name',
       'country_name', '_geoloc', 'popularity', 'length', 'elevation_gain',
       'difficulty_rating', 'route_type', 'visitor_usage', 'avg_rating',
       'num_reviews', 'features', 'activities', 'units'],
      dtype='object')

In [29]:
df_out.columns.name

In [30]:
df_out.index.name

In [34]:
df_true = pd.DataFrame(
    [[10008848, 'Baxter Creek Trail',
        'Great Smoky Mountains National Park', 'Waynesville',
        'North Carolina', 'United States',
        "{'lat': 35.75111, 'lng': -83.10966}", 4.9204, 14.36, 1522.7808,
        5, 'point to point', 2.0, 4.5, 7,
        "['dogs-no', 'forest', 'views', 'wild-flowers']",
        "['birding', 'hiking', 'nature-trips']", 'i'],
       [10032315, 'Sunnybrook Meadows Trail', 'Olympic National Park',
        'Brinnon', 'Washington', 'United States',
        "{'lat': 47.72974, 'lng': -123.14179}", 3.0804, 15.0, 2102.8152,
        7, 'out and back', 2.0, 5.0, 1,
        "['dogs-no', 'views', 'wild-flowers', 'wildlife']",
        "['backpacking', 'birding', 'camping', 'hiking', 'nature-trips']",
        'i'],
       [10027503, 'Clouds Rest Trail via Tenaya Lake',
        'Yosemite National Park', 'Yosemite Valley', 'California',
        'United States', "{'lat': 37.82585, 'lng': -119.47046}", 38.5795,
        11.25, 948.8424, 5, 'out and back', 3.0, 5.0, 685,
        "['dogs-no', 'forest', 'lake', 'views', 'wild-flowers', 'wildlife']",
        "['backpacking', 'birding', 'camping', 'hiking', 'nature-trips', 'trail-running', 'horseback-riding']",
        'i']],
index=[1698, 2919, 224],
columns=['trail_id', 'name', 'area_name', 'city_name', 'state_name',
       'country_name', '_geoloc', 'popularity', 'length', 'elevation_gain',
       'difficulty_rating', 'route_type', 'visitor_usage', 'avg_rating',
       'num_reviews', 'features', 'activities', 'units'])

In [35]:
df_true.equals(df_out)

True

In [36]:
def test_yards_to_miles():
    # Define input dataframe
    df_in_values = [[10008848, 'Baxter Creek Trail',
        'Great Smoky Mountains National Park', 'Waynesville',
        'North Carolina', 'United States',
        "{'lat': 35.75111, 'lng': -83.10966}", 4.9204, 25266.638,
        1522.7808, 5, 'point to point', 2.0, 4.5, 7,
        "['dogs-no', 'forest', 'views', 'wild-flowers']",
        "['birding', 'hiking', 'nature-trips']", 'i'],
       [10032315, 'Sunnybrook Meadows Trail', 'Olympic National Park',
        'Brinnon', 'Washington', 'United States',
        "{'lat': 47.72974, 'lng': -123.14179}", 3.0804, 26393.176,
        2102.8152, 7, 'out and back', 2.0, 5.0, 1,
        "['dogs-no', 'views', 'wild-flowers', 'wildlife']",
        "['backpacking', 'birding', 'camping', 'hiking', 'nature-trips']",
        'i'],
       [10027503, 'Clouds Rest Trail via Tenaya Lake',
        'Yosemite National Park', 'Yosemite Valley', 'California',
        'United States', "{'lat': 37.82585, 'lng': -119.47046}", 38.5795,
        19794.882, 948.8424, 5, 'out and back', 3.0, 5.0, 685,
        "['dogs-no', 'forest', 'lake', 'views', 'wild-flowers', 'wildlife']",
        "['backpacking', 'birding', 'camping', 'hiking', 'nature-trips', 'trail-running', 'horseback-riding']",
        'i']]
    df_in_index = [1698, 2919, 224]
    df_in_columns = ['trail_id', 'name', 'area_name', 'city_name', 'state_name',
           'country_name', '_geoloc', 'popularity', 'length', 'elevation_gain',
           'difficulty_rating', 'route_type', 'visitor_usage', 'avg_rating',
           'num_reviews', 'features', 'activities', 'units']
    df_in = pd.DataFrame(df_in_values, index=df_in_index, columns=df_in_columns)
    
    # Define expected output
    df_true = pd.DataFrame(
    [[10008848, 'Baxter Creek Trail',
        'Great Smoky Mountains National Park', 'Waynesville',
        'North Carolina', 'United States',
        "{'lat': 35.75111, 'lng': -83.10966}", 4.9204, 14.36, 1522.7808,
        5, 'point to point', 2.0, 4.5, 7,
        "['dogs-no', 'forest', 'views', 'wild-flowers']",
        "['birding', 'hiking', 'nature-trips']", 'i'],
       [10032315, 'Sunnybrook Meadows Trail', 'Olympic National Park',
        'Brinnon', 'Washington', 'United States',
        "{'lat': 47.72974, 'lng': -123.14179}", 3.0804, 15.0, 2102.8152,
        7, 'out and back', 2.0, 5.0, 1,
        "['dogs-no', 'views', 'wild-flowers', 'wildlife']",
        "['backpacking', 'birding', 'camping', 'hiking', 'nature-trips']",
        'i'],
       [10027503, 'Clouds Rest Trail via Tenaya Lake',
        'Yosemite National Park', 'Yosemite Valley', 'California',
        'United States', "{'lat': 37.82585, 'lng': -119.47046}", 38.5795,
        11.25, 948.8424, 5, 'out and back', 3.0, 5.0, 685,
        "['dogs-no', 'forest', 'lake', 'views', 'wild-flowers', 'wildlife']",
        "['backpacking', 'birding', 'camping', 'hiking', 'nature-trips', 'trail-running', 'horseback-riding']",
        'i']],
    index=[1698, 2919, 224],
    columns=['trail_id', 'name', 'area_name', 'city_name', 'state_name',
           'country_name', '_geoloc', 'popularity', 'length', 'elevation_gain',
           'difficulty_rating', 'route_type', 'visitor_usage', 'avg_rating',
           'num_reviews', 'features', 'activities', 'units'])
    
    # Create test output
    df_test = yards_to_miles(df_in, 'length', 2)
    
    assert df_test.equals(df_true)

In [38]:
test_yards_to_miles()

Unhappy path: feature doesn't exsist

In [40]:
def test_yards_to_miles_key_error():
    # Define input dataframe
    df_in_values = [[10008848, 'Baxter Creek Trail',
        'Great Smoky Mountains National Park', 'Waynesville',
        'North Carolina', 'United States',
        "{'lat': 35.75111, 'lng': -83.10966}", 4.9204, 25266.638,
        1522.7808, 5, 'point to point', 2.0, 4.5, 7,
        "['dogs-no', 'forest', 'views', 'wild-flowers']",
        "['birding', 'hiking', 'nature-trips']", 'i'],
       [10032315, 'Sunnybrook Meadows Trail', 'Olympic National Park',
        'Brinnon', 'Washington', 'United States',
        "{'lat': 47.72974, 'lng': -123.14179}", 3.0804, 26393.176,
        2102.8152, 7, 'out and back', 2.0, 5.0, 1,
        "['dogs-no', 'views', 'wild-flowers', 'wildlife']",
        "['backpacking', 'birding', 'camping', 'hiking', 'nature-trips']",
        'i'],
       [10027503, 'Clouds Rest Trail via Tenaya Lake',
        'Yosemite National Park', 'Yosemite Valley', 'California',
        'United States', "{'lat': 37.82585, 'lng': -119.47046}", 38.5795,
        19794.882, 948.8424, 5, 'out and back', 3.0, 5.0, 685,
        "['dogs-no', 'forest', 'lake', 'views', 'wild-flowers', 'wildlife']",
        "['backpacking', 'birding', 'camping', 'hiking', 'nature-trips', 'trail-running', 'horseback-riding']",
        'i']]
    df_in_index = [1698, 2919, 224]
    df_in_columns = ['trail_id', 'name', 'area_name', 'city_name', 'state_name',
           'country_name', '_geoloc', 'popularity', 'length', 'elevation_gain',
           'difficulty_rating', 'route_type', 'visitor_usage', 'avg_rating',
           'num_reviews', 'features', 'activities', 'units']
    df_in = pd.DataFrame(df_in_values, index=df_in_index, columns=df_in_columns)
    
    # Define expected output
    with pytest.raises(KeyError):
        yards_to_miles(df_in, 'error', 2)

In [43]:
test_yards_to_miles_key_error()

Unhappy path: not a dataframe

In [45]:
def test_yards_to_miles_non_df():
    df_in = 'I am not a dataframe'
    
    with pytest.raises(TypeError):
        yards_to_miles(df_in, 'length', 2)

In [46]:
test_multiply_feature_non_df()

## Create tests for `df_drop_str()`

In [49]:
df_in_values = [[10008848, 'Baxter Creek Trail',
        'Great Smoky Mountains National Park', 'Waynesville',
        'North Carolina', 'United States',
        "{'lat': 35.75111, 'lng': -83.10966}", 4.9204, 25266.638,
        1522.7808, 5, 'point to point', 2.0, 4.5, 7,
        "['dogs-no', 'forest', 'views', 'wild-flowers']",
        "['birding', 'hiking', 'nature-trips']", 'i'],
       [10032315, 'Sunnybrook Meadows Trail', 'Olympic National Park',
        'Brinnon', 'Washington', 'United States',
        "{'lat': 47.72974, 'lng': -123.14179}", 3.0804, 26393.176,
        2102.8152, 7, 'out and back', 2.0, 5.0, 1,
        "['dogs-no', 'views', 'wild-flowers', 'wildlife']",
        "['backpacking', 'birding', 'camping', 'hiking', 'nature-trips']",
        'i'],
       [10027503, 'Clouds Rest Trail via Tenaya Lake',
        'Yosemite National Park', 'Yosemite Valley', 'California',
        'United States', "{'lat': 37.82585, 'lng': -119.47046}", 38.5795,
        19794.882, 948.8424, 5, 'out and back', 3.0, 5.0, 685,
        "['dogs-no', 'forest', 'lake', 'views', 'wild-flowers', 'wildlife']",
        "['backpacking', 'birding', 'camping', 'hiking', 'nature-trips', 'trail-running', 'horseback-riding']",
        'i']]
df_in_index = [1698, 2919, 224]
df_in_columns = ['trail_id', 'name', 'area_name', 'city_name', 'state_name',
       'country_name', '_geoloc', 'popularity', 'length', 'elevation_gain',
       'difficulty_rating', 'route_type', 'visitor_usage', 'avg_rating',
       'num_reviews', 'features', 'activities', 'units']
df_in = pd.DataFrame(df_in_values, index=df_in_index, columns=df_in_columns)

In [50]:
df_out = df_drop_str(df_in, 'name', 'closed')
df_out

,trail_id,name,area_name,city_name,state_name,country_name,_geoloc,popularity,length,elevation_gain,difficulty_rating,route_type,visitor_usage,avg_rating,num_reviews,features,activities,units
1698,10008848,Baxter Creek Trail,Great Smoky Mountains National Park,Waynesville,North Carolina,United States,"{'lat': 35.75111, 'lng': -83.10966}",4.9204,25266.638,1522.7808,5,point to point,2.0,4.5,7,"['dogs-no', 'forest', 'views', 'wild-flowers']","['birding', 'hiking', 'nature-trips']",i
2919,10032315,Sunnybrook Meadows Trail,Olympic National Park,Brinnon,Washington,United States,"{'lat': 47.72974, 'lng': -123.14179}",3.0804,26393.176,2102.8152,7,out and back,2.0,5.0,1,"['dogs-no', 'views', 'wild-flowers', 'wildlife']","['backpacking', 'birding', 'camping', 'hiking'...",i
224,10027503,Clouds Rest Trail via Tenaya Lake,Yosemite National Park,Yosemite Valley,California,United States,"{'lat': 37.82585, 'lng': -119.47046}",38.5795,19794.882,948.8424,5,out and back,3.0,5.0,685,"['dogs-no', 'forest', 'lake', 'views', 'wild-f...","['backpacking', 'birding', 'camping', 'hiking'...",i


In [51]:
df_out.values

array([[10008848, 'Baxter Creek Trail',
        'Great Smoky Mountains National Park', 'Waynesville',
        'North Carolina', 'United States',
        "{'lat': 35.75111, 'lng': -83.10966}", 4.9204, 25266.638,
        1522.7808, 5, 'point to point', 2.0, 4.5, 7,
        "['dogs-no', 'forest', 'views', 'wild-flowers']",
        "['birding', 'hiking', 'nature-trips']", 'i'],
       [10032315, 'Sunnybrook Meadows Trail', 'Olympic National Park',
        'Brinnon', 'Washington', 'United States',
        "{'lat': 47.72974, 'lng': -123.14179}", 3.0804, 26393.176,
        2102.8152, 7, 'out and back', 2.0, 5.0, 1,
        "['dogs-no', 'views', 'wild-flowers', 'wildlife']",
        "['backpacking', 'birding', 'camping', 'hiking', 'nature-trips']",
        'i'],
       [10027503, 'Clouds Rest Trail via Tenaya Lake',
        'Yosemite National Park', 'Yosemite Valley', 'California',
        'United States', "{'lat': 37.82585, 'lng': -119.47046}", 38.5795,
        19794.882, 948.8424, 5, 'out an

In [52]:
df_out.index

Int64Index([1698, 2919, 224], dtype='int64')

In [53]:
df_out.columns

Index(['trail_id', 'name', 'area_name', 'city_name', 'state_name',
       'country_name', '_geoloc', 'popularity', 'length', 'elevation_gain',
       'difficulty_rating', 'route_type', 'visitor_usage', 'avg_rating',
       'num_reviews', 'features', 'activities', 'units'],
      dtype='object')

In [54]:
df_out.columns.name

In [55]:
df_out.index.name

In [56]:
df_true = pd.DataFrame(
    [[10008848, 'Baxter Creek Trail',
        'Great Smoky Mountains National Park', 'Waynesville',
        'North Carolina', 'United States',
        "{'lat': 35.75111, 'lng': -83.10966}", 4.9204, 25266.638,
        1522.7808, 5, 'point to point', 2.0, 4.5, 7,
        "['dogs-no', 'forest', 'views', 'wild-flowers']",
        "['birding', 'hiking', 'nature-trips']", 'i'],
       [10032315, 'Sunnybrook Meadows Trail', 'Olympic National Park',
        'Brinnon', 'Washington', 'United States',
        "{'lat': 47.72974, 'lng': -123.14179}", 3.0804, 26393.176,
        2102.8152, 7, 'out and back', 2.0, 5.0, 1,
        "['dogs-no', 'views', 'wild-flowers', 'wildlife']",
        "['backpacking', 'birding', 'camping', 'hiking', 'nature-trips']",
        'i'],
       [10027503, 'Clouds Rest Trail via Tenaya Lake',
        'Yosemite National Park', 'Yosemite Valley', 'California',
        'United States', "{'lat': 37.82585, 'lng': -119.47046}", 38.5795,
        19794.882, 948.8424, 5, 'out and back', 3.0, 5.0, 685,
        "['dogs-no', 'forest', 'lake', 'views', 'wild-flowers', 'wildlife']",
        "['backpacking', 'birding', 'camping', 'hiking', 'nature-trips', 'trail-running', 'horseback-riding']",
        'i']],
index=[1698, 2919, 224],
columns=['trail_id', 'name', 'area_name', 'city_name', 'state_name',
       'country_name', '_geoloc', 'popularity', 'length', 'elevation_gain',
       'difficulty_rating', 'route_type', 'visitor_usage', 'avg_rating',
       'num_reviews', 'features', 'activities', 'units'])

In [57]:
df_true.equals(df_out)

True

In [58]:
def test_df_drop_str():
    
    # Define input dataframe
    df_in_values = [[10008848, 'Baxter Creek Trail',
        'Great Smoky Mountains National Park', 'Waynesville',
        'North Carolina', 'United States',
        "{'lat': 35.75111, 'lng': -83.10966}", 4.9204, 25266.638,
        1522.7808, 5, 'point to point', 2.0, 4.5, 7,
        "['dogs-no', 'forest', 'views', 'wild-flowers']",
        "['birding', 'hiking', 'nature-trips']", 'i'],
       [10032315, 'Sunnybrook Meadows Trail', 'Olympic National Park',
        'Brinnon', 'Washington', 'United States',
        "{'lat': 47.72974, 'lng': -123.14179}", 3.0804, 26393.176,
        2102.8152, 7, 'out and back', 2.0, 5.0, 1,
        "['dogs-no', 'views', 'wild-flowers', 'wildlife']",
        "['backpacking', 'birding', 'camping', 'hiking', 'nature-trips']",
        'i'],
       [10027503, 'Clouds Rest Trail via Tenaya Lake',
        'Yosemite National Park', 'Yosemite Valley', 'California',
        'United States', "{'lat': 37.82585, 'lng': -119.47046}", 38.5795,
        19794.882, 948.8424, 5, 'out and back', 3.0, 5.0, 685,
        "['dogs-no', 'forest', 'lake', 'views', 'wild-flowers', 'wildlife']",
        "['backpacking', 'birding', 'camping', 'hiking', 'nature-trips', 'trail-running', 'horseback-riding']",
        'i']]
    df_in_index = [1698, 2919, 224]
    df_in_columns = ['trail_id', 'name', 'area_name', 'city_name', 'state_name',
           'country_name', '_geoloc', 'popularity', 'length', 'elevation_gain',
           'difficulty_rating', 'route_type', 'visitor_usage', 'avg_rating',
           'num_reviews', 'features', 'activities', 'units']
    df_in = pd.DataFrame(df_in_values, index=df_in_index, columns=df_in_columns)
    
    # Define expected output
    df_true = pd.DataFrame(
        [[10008848, 'Baxter Creek Trail',
            'Great Smoky Mountains National Park', 'Waynesville',
            'North Carolina', 'United States',
            "{'lat': 35.75111, 'lng': -83.10966}", 4.9204, 25266.638,
            1522.7808, 5, 'point to point', 2.0, 4.5, 7,
            "['dogs-no', 'forest', 'views', 'wild-flowers']",
            "['birding', 'hiking', 'nature-trips']", 'i'],
           [10032315, 'Sunnybrook Meadows Trail', 'Olympic National Park',
            'Brinnon', 'Washington', 'United States',
            "{'lat': 47.72974, 'lng': -123.14179}", 3.0804, 26393.176,
            2102.8152, 7, 'out and back', 2.0, 5.0, 1,
            "['dogs-no', 'views', 'wild-flowers', 'wildlife']",
            "['backpacking', 'birding', 'camping', 'hiking', 'nature-trips']",
            'i'],
           [10027503, 'Clouds Rest Trail via Tenaya Lake',
            'Yosemite National Park', 'Yosemite Valley', 'California',
            'United States', "{'lat': 37.82585, 'lng': -119.47046}", 38.5795,
            19794.882, 948.8424, 5, 'out and back', 3.0, 5.0, 685,
            "['dogs-no', 'forest', 'lake', 'views', 'wild-flowers', 'wildlife']",
            "['backpacking', 'birding', 'camping', 'hiking', 'nature-trips', 'trail-running', 'horseback-riding']",
            'i']],
    index=[1698, 2919, 224],
    columns=['trail_id', 'name', 'area_name', 'city_name', 'state_name',
           'country_name', '_geoloc', 'popularity', 'length', 'elevation_gain',
           'difficulty_rating', 'route_type', 'visitor_usage', 'avg_rating',
           'num_reviews', 'features', 'activities', 'units'])
    
    # Create test output
    df_test = df_drop_str(df_in, 'name', 'closed')
    
    assert df_test.equals(df_true)

In [59]:
test_df_drop_str()

Unhappy path: feature doesn't exsist

In [60]:
def test_df_drop_str_key_error():
    # Define input dataframe
    df_in_values = [[10008848, 'Baxter Creek Trail',
        'Great Smoky Mountains National Park', 'Waynesville',
        'North Carolina', 'United States',
        "{'lat': 35.75111, 'lng': -83.10966}", 4.9204, 25266.638,
        1522.7808, 5, 'point to point', 2.0, 4.5, 7,
        "['dogs-no', 'forest', 'views', 'wild-flowers']",
        "['birding', 'hiking', 'nature-trips']", 'i'],
       [10032315, 'Sunnybrook Meadows Trail', 'Olympic National Park',
        'Brinnon', 'Washington', 'United States',
        "{'lat': 47.72974, 'lng': -123.14179}", 3.0804, 26393.176,
        2102.8152, 7, 'out and back', 2.0, 5.0, 1,
        "['dogs-no', 'views', 'wild-flowers', 'wildlife']",
        "['backpacking', 'birding', 'camping', 'hiking', 'nature-trips']",
        'i'],
       [10027503, 'Clouds Rest Trail via Tenaya Lake',
        'Yosemite National Park', 'Yosemite Valley', 'California',
        'United States', "{'lat': 37.82585, 'lng': -119.47046}", 38.5795,
        19794.882, 948.8424, 5, 'out and back', 3.0, 5.0, 685,
        "['dogs-no', 'forest', 'lake', 'views', 'wild-flowers', 'wildlife']",
        "['backpacking', 'birding', 'camping', 'hiking', 'nature-trips', 'trail-running', 'horseback-riding']",
        'i']]
    df_in_index = [1698, 2919, 224]
    df_in_columns = ['trail_id', 'name', 'area_name', 'city_name', 'state_name',
           'country_name', '_geoloc', 'popularity', 'length', 'elevation_gain',
           'difficulty_rating', 'route_type', 'visitor_usage', 'avg_rating',
           'num_reviews', 'features', 'activities', 'units']
    df_in = pd.DataFrame(df_in_values, index=df_in_index, columns=df_in_columns)
    
    # Define expected output
    with pytest.raises(KeyError):
        df_drop_str(df_in, 'error', 'closed')

In [61]:
test_df_drop_str_key_error()

Unhappy path: not a dataframe

In [62]:
def test_df_drop_str_non_df():
    df_in = 'I am not a dataframe'
    
    with pytest.raises(TypeError):
        df_drop_str(df_in, 'name', 'closed')

In [63]:
test_df_drop_str_non_df()